In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pickle
import math
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:

df_gold = pd.read_csv("/content/drive/MyDrive/NN project/Gold Futures Historical Data.csv")
df_ptt = pd.read_csv("/content/drive/MyDrive/NN project/PTT Historical Data.csv")
df_pttep = pd.read_csv("/content/drive/MyDrive/NN project/PTTEP Historical Data.csv")
df_top = pd.read_csv("/content/drive/MyDrive/NN project/TOP Historical Data.csv")
df_usDollar = pd.read_csv('/content/drive/MyDrive/NN project/US Dollar Index Historical Data.csv')


In [4]:
df_gold.drop(columns=['Open','High' ,'Low','Vol.','Change %'],inplace=True)
df_ptt.drop(columns=['Open','High' ,'Low','Vol.','Change %'],inplace=True)
df_pttep.drop(columns=['Open','High' ,'Low','Vol.','Change %'],inplace=True)
df_top.drop(columns=['Open','High' ,'Low','Vol.','Change %'],inplace=True)
df_usDollar.drop(columns=['Open','High' ,'Low','Vol.','Change %'],inplace=True)

In [5]:
# drop row ซ้ำ
def dropDuplicate( df_gold , df_ptt , df_pttep,df_top,df_usDollar ) :
  df_gold.drop_duplicates(subset=['date_time'],inplace=True)
  df_ptt.drop_duplicates(subset=['date_time'], inplace=True)
  df_pttep.drop_duplicates(subset=['date_time'], inplace=True)
  df_top.drop_duplicates(subset=['date_time'], inplace=True)
  df_usDollar.drop_duplicates(subset=['date_time'], inplace=True)
  return [df_gold,df_ptt,df_pttep,df_top,df_usDollar]

In [6]:
# เปลี่ยนชื่อ  date_time
def rename( df_gold , df_ptt , df_pttep , df_top , df_usDollar ):
  df_gold.rename({df_gold.columns.tolist()[0]: 'date_time'}, axis=1, inplace=True)
  df_ptt.rename({df_ptt.columns.tolist()[0]: 'date_time'}, axis=1, inplace=True)
  df_pttep.rename({df_pttep.columns.tolist()[0]: 'date_time'}, axis=1, inplace=True)
  df_top.rename({df_top.columns.tolist()[0]: 'date_time'}, axis=1, inplace=True)
  df_usDollar.rename({df_usDollar.columns.tolist()[0]: 'date_time'}, axis=1, inplace=True)
  return [df_gold,df_ptt,df_pttep,df_top,df_usDollar]

In [7]:
rename(df_gold , df_ptt , df_pttep , df_top , df_usDollar )

[         date_time     Price
 0     Jan 01, 2021  1,901.60
 1     Dec 31, 2020  1,895.10
 2     Dec 30, 2020  1,893.40
 3     Dec 29, 2020  1,880.50
 4     Dec 28, 2020  1,877.90
 ...            ...       ...
 1583  Jan 07, 2015  1,353.60
 1584  Jan 06, 2015  1,362.20
 1585  Jan 05, 2015  1,353.50
 1586  Jan 02, 2015  1,337.70
 1587  Jan 01, 2015  1,186.80
 
 [1588 rows x 2 columns],          date_time  Price
 0     Dec 30, 2020  42.50
 1     Dec 29, 2020  42.50
 2     Dec 28, 2020  41.00
 3     Dec 25, 2020  42.00
 4     Dec 24, 2020  41.25
 ...            ...    ...
 1458  Jan 09, 2015  33.80
 1459  Jan 08, 2015  34.00
 1460  Jan 07, 2015  33.40
 1461  Jan 06, 2015  32.30
 1462  Jan 05, 2015  31.50
 
 [1463 rows x 2 columns],          date_time   Price
 0     Dec 30, 2020   98.25
 1     Dec 29, 2020   97.75
 2     Dec 28, 2020   95.00
 3     Dec 25, 2020   96.50
 4     Dec 24, 2020   95.75
 ...            ...     ...
 1458  Jan 09, 2015  112.50
 1459  Jan 08, 2015  110.00
 1460  Jan

In [8]:
# จัด format dateTime 
def convert_month_day_year(x):
  [month_date,year] = x.strip().split(",")
  month_date = month_date.strip()
  year = year.strip()
  [month,day] = month_date.split(' ')
  month = month.strip()
  day = day.strip()
  if(month == "Jan"):
    month = "01"
  elif(month == "Feb"):
    month = "02"
  elif(month == "Mar"):
    month = "03"
  elif(month == "Apr"):
    month = "04"
  elif(month == "May"):
    month = "05"
  elif(month == "Jun"):
    month = "06"
  elif(month == "Jul"):
    month = "07"
  elif(month == "Aug"):
    month = "08"
  elif(month == "Sep"):
    month = "09"
  elif(month == "Oct"):
    month = "10"
  elif(month == "Nov"):
    month = "11"
  elif(month == "Dec"):
    month = "12"
  time = " 00:00:00"
  newDate = year+ "-" + month + "-" + day + " " + time
  return newDate

In [9]:
#จัดรูปแบบ date_time ใหม่(date/month/year)
def formatDate( df_gold , df_ptt , df_top ,df_pttep,df_usDollar, fotmatDateFunction ):
  df_gold['date_time']    =  df_gold['date_time'].apply(fotmatDateFunction)
  df_ptt['date_time'] =  df_ptt['date_time'].apply(fotmatDateFunction)
  df_pttep["date_time"] = df_pttep['date_time'].apply(fotmatDateFunction)
  df_top['date_time']  = df_top['date_time'].apply(fotmatDateFunction)
  df_usDollar["date_time"] = df_usDollar['date_time'].apply(fotmatDateFunction)
  return [df_gold,df_ptt,df_pttep,df_top,df_usDollar]

In [10]:
def toDate(df_gold , df_ptt , df_top ,df_pttep,df_usDollar):

  df_gold['date_time'] = pd.to_datetime(df_gold['date_time'])
  df_gold.set_index('date_time', inplace=True)

  df_ptt['date_time'] = pd.to_datetime(df_ptt['date_time'])
  df_ptt.set_index('date_time', inplace=True)

  df_top['date_time'] = pd.to_datetime(df_top['date_time'])
  df_top.set_index('date_time', inplace=True)

  df_pttep['date_time'] = pd.to_datetime(df_pttep['date_time'])
  df_pttep.set_index('date_time', inplace=True)

  df_usDollar['date_time'] = pd.to_datetime(df_usDollar['date_time'])
  df_usDollar.set_index('date_time', inplace=True)

  return [df_gold,df_ptt,df_pttep,df_top,df_usDollar]

In [11]:
dropDuplicate(df_gold,df_ptt,df_pttep,df_top,df_usDollar)

[         date_time     Price
 0     Jan 01, 2021  1,901.60
 1     Dec 31, 2020  1,895.10
 2     Dec 30, 2020  1,893.40
 3     Dec 29, 2020  1,880.50
 4     Dec 28, 2020  1,877.90
 ...            ...       ...
 1583  Jan 07, 2015  1,353.60
 1584  Jan 06, 2015  1,362.20
 1585  Jan 05, 2015  1,353.50
 1586  Jan 02, 2015  1,337.70
 1587  Jan 01, 2015  1,186.80
 
 [1588 rows x 2 columns],          date_time  Price
 0     Dec 30, 2020  42.50
 1     Dec 29, 2020  42.50
 2     Dec 28, 2020  41.00
 3     Dec 25, 2020  42.00
 4     Dec 24, 2020  41.25
 ...            ...    ...
 1458  Jan 09, 2015  33.80
 1459  Jan 08, 2015  34.00
 1460  Jan 07, 2015  33.40
 1461  Jan 06, 2015  32.30
 1462  Jan 05, 2015  31.50
 
 [1463 rows x 2 columns],          date_time   Price
 0     Dec 30, 2020   98.25
 1     Dec 29, 2020   97.75
 2     Dec 28, 2020   95.00
 3     Dec 25, 2020   96.50
 4     Dec 24, 2020   95.75
 ...            ...     ...
 1458  Jan 09, 2015  112.50
 1459  Jan 08, 2015  110.00
 1460  Jan

In [12]:
formatDate(df_gold , df_ptt , df_top ,df_pttep,df_usDollar, convert_month_day_year)

[                 date_time     Price
 0     2021-01-01  00:00:00  1,901.60
 1     2020-12-31  00:00:00  1,895.10
 2     2020-12-30  00:00:00  1,893.40
 3     2020-12-29  00:00:00  1,880.50
 4     2020-12-28  00:00:00  1,877.90
 ...                    ...       ...
 1583  2015-01-07  00:00:00  1,353.60
 1584  2015-01-06  00:00:00  1,362.20
 1585  2015-01-05  00:00:00  1,353.50
 1586  2015-01-02  00:00:00  1,337.70
 1587  2015-01-01  00:00:00  1,186.80
 
 [1588 rows x 2 columns],                  date_time  Price
 0     2020-12-30  00:00:00  42.50
 1     2020-12-29  00:00:00  42.50
 2     2020-12-28  00:00:00  41.00
 3     2020-12-25  00:00:00  42.00
 4     2020-12-24  00:00:00  41.25
 ...                    ...    ...
 1458  2015-01-09  00:00:00  33.80
 1459  2015-01-08  00:00:00  34.00
 1460  2015-01-07  00:00:00  33.40
 1461  2015-01-06  00:00:00  32.30
 1462  2015-01-05  00:00:00  31.50
 
 [1463 rows x 2 columns],                  date_time   Price
 0     2020-12-30  00:00:00   98.2

In [13]:
df_gold

,date_time,Price
0,2021-01-01 00:00:00,"1,901.60"
1,2020-12-31 00:00:00,"1,895.10"
2,2020-12-30 00:00:00,"1,893.40"
3,2020-12-29 00:00:00,"1,880.50"
4,2020-12-28 00:00:00,"1,877.90"
...,...,...
1583,2015-01-07 00:00:00,"1,353.60"
1584,2015-01-06 00:00:00,"1,362.20"
1585,2015-01-05 00:00:00,"1,353.50"
1586,2015-01-02 00:00:00,"1,337.70"


In [14]:
toDate(df_gold , df_ptt , df_top ,df_pttep,df_usDollar)

[               Price
 date_time           
 2021-01-01  1,901.60
 2020-12-31  1,895.10
 2020-12-30  1,893.40
 2020-12-29  1,880.50
 2020-12-28  1,877.90
 ...              ...
 2015-01-07  1,353.60
 2015-01-06  1,362.20
 2015-01-05  1,353.50
 2015-01-02  1,337.70
 2015-01-01  1,186.80
 
 [1588 rows x 1 columns],             Price
 date_time        
 2020-12-30  42.50
 2020-12-29  42.50
 2020-12-28  41.00
 2020-12-25  42.00
 2020-12-24  41.25
 ...           ...
 2015-01-09  33.80
 2015-01-08  34.00
 2015-01-07  33.40
 2015-01-06  32.30
 2015-01-05  31.50
 
 [1463 rows x 1 columns],              Price
 date_time         
 2020-12-30   98.25
 2020-12-29   97.75
 2020-12-28   95.00
 2020-12-25   96.50
 2020-12-24   95.75
 ...            ...
 2015-01-09  112.50
 2015-01-08  110.00
 2015-01-07  106.50
 2015-01-06  107.00
 2015-01-05  107.00
 
 [1463 rows x 1 columns],             Price
 date_time        
 2020-12-30  52.00
 2020-12-29  53.25
 2020-12-28  51.25
 2020-12-25  54.00
 2020-12-24 

In [15]:
# เรียง row ตาม date_time
def sortDateTime( df_gold , df_ptt , df_top ,df_pttep,df_usDollar) : 
  df_gold.sort_values(by='date_time', inplace=True)
  df_ptt.sort_values(by='date_time', inplace=True)
  df_top.sort_values(by='date_time', inplace=True)
  df_pttep.sort_values(by='date_time', inplace=True)
  df_usDollar.sort_values(by='date_time', inplace=True)
  return [df_gold,df_ptt,df_pttep,df_top,df_usDollar]

In [16]:
sortDateTime( df_gold , df_ptt , df_top ,df_pttep,df_usDollar)

[               Price
 date_time           
 2015-01-01  1,186.80
 2015-01-02  1,337.70
 2015-01-05  1,353.50
 2015-01-06  1,362.20
 2015-01-07  1,353.60
 ...              ...
 2020-12-28  1,877.90
 2020-12-29  1,880.50
 2020-12-30  1,893.40
 2020-12-31  1,895.10
 2021-01-01  1,901.60
 
 [1588 rows x 1 columns],             Price
 date_time        
 2015-01-05  31.50
 2015-01-06  32.30
 2015-01-07  33.40
 2015-01-08  34.00
 2015-01-09  33.80
 ...           ...
 2020-12-24  41.25
 2020-12-25  42.00
 2020-12-28  41.00
 2020-12-29  42.50
 2020-12-30  42.50
 
 [1463 rows x 1 columns],              Price
 date_time         
 2015-01-05  107.00
 2015-01-06  107.00
 2015-01-07  106.50
 2015-01-08  110.00
 2015-01-09  112.50
 ...            ...
 2020-12-24   95.75
 2020-12-25   96.50
 2020-12-28   95.00
 2020-12-29   97.75
 2020-12-30   98.25
 
 [1463 rows x 1 columns],             Price
 date_time        
 2015-01-05  42.75
 2015-01-06  43.00
 2015-01-07  44.25
 2015-01-08  45.75
 2015-01-09 

In [17]:
df_gold['Price'] = df_gold['Price'].str.replace(',', '').astype(float)

In [18]:
# นำ box-plot มาใช้โดยที่ data < q1 -> q1   , data > q3 -> q3
def removeOutlier(df):
  cols = df.columns.tolist()[1:-1:]
  for col in cols:
    q1 = df[col].quantile(q=0.25)
    q3 = df[col].quantile(q=0.75)
    IQR = q3 - q1
    df[df[col] > q3 + 1.5*IQR] = q3
    df[df[col] < q1 - 1.5*IQR] = q1
  return df

In [19]:
removeOutlier(df_gold)
removeOutlier(df_ptt)
removeOutlier(df_pttep)
removeOutlier(df_top)
removeOutlier(df_usDollar)

,Price
date_time,
2015-01-01,90.27
2015-01-02,91.08
2015-01-05,91.38
2015-01-06,91.50
2015-01-07,91.89
...,...
2020-12-28,90.34
2020-12-29,89.99
2020-12-30,89.68


In [20]:
df_gold = df_gold.resample('1h').ffill()
df_gold = df_gold.reindex(pd.date_range(start="2015-01-05 00:00:00", end= "2020-12-30 00:00:00", freq="24h"), fill_value=np.nan)
df_gold = df_gold.interpolate(method='nearest')

df_ptt = df_ptt.resample('1h').ffill()
df_ptt = df_ptt.reindex(pd.date_range(start="2015-01-05 00:00:00", end= "2020-12-30 00:00:00", freq="24h"), fill_value=np.nan)
df_ptt = df_ptt.interpolate(method='nearest')

df_pttep = df_pttep.resample('1h').ffill()
df_pttep = df_pttep.reindex(pd.date_range(start="2015-01-05 00:00:00", end= "2020-12-30 00:00:00", freq="24h"), fill_value=np.nan)
df_pttep = df_pttep.interpolate(method='nearest')

df_top = df_top.resample('1h').ffill()
df_top = df_top.reindex(pd.date_range(start="2015-01-05 00:00:00", end= "2020-12-30 00:00:00", freq="24h"), fill_value=np.nan)
df_top = df_top.interpolate(method='nearest')

df_usDollar = df_usDollar.resample('1h').ffill()
df_usDollar = df_usDollar.reindex(pd.date_range(start="2015-01-05 00:00:00", end= "2020-12-30 00:00:00", freq="24h"), fill_value=np.nan)
df_usDollar = df_usDollar.interpolate(method='nearest')

In [21]:
# retrieveData(df_gold , df_ptt , df_top ,df_pttep , df_usDollar , "2015-01-05 00:00:00" , "2020-12-30 00:00:00" , "24h") 

In [22]:
null_counts = df_gold.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

Number of null values in each column:
Price    0
dtype: int64


In [23]:
null_counts = df_ptt.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

Number of null values in each column:
Price    0
dtype: int64


In [24]:
null_counts = df_top.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

Number of null values in each column:
Price    0
dtype: int64


In [25]:
null_counts = df_pttep.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

Number of null values in each column:
Price    0
dtype: int64


In [26]:
null_counts = df_usDollar.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

Number of null values in each column:
Price    0
dtype: int64


In [27]:
df_gold.shape

(2187, 1)

In [28]:
df_ptt.shape

(2187, 1)

In [29]:
df_pttep.shape

(2187, 1)

In [30]:
df_usDollar.shape

(2187, 1)

In [31]:
df_top.shape

(2187, 1)

In [32]:
df_gold.rename(columns = {'Price':'gold_price'}, inplace = True)
df_ptt.rename(columns = {'Price':'ptt_price'}, inplace = True)
df_pttep.rename(columns = {'Price':'pttep_price'}, inplace = True)
df_top.rename(columns = {'Price':'top_price'}, inplace = True)
df_usDollar.rename(columns = {'Price':'usDollar_price'}, inplace = True)

In [33]:
df_gold.index.name = 'date_time'
df_ptt.index.name = 'date_time'
df_pttep.index.name = 'date_time'
df_top.index.name = 'date_time'
df_usDollar.index.name = 'date_time'

df_ptt.reset_index(inplace=True)
df_pttep.reset_index(inplace=True)
df_gold.reset_index(inplace=True)
df_top.reset_index(inplace=True)
df_usDollar.reset_index(inplace=True)


df_gold.drop(labels=['date_time'], axis=1, inplace=True)
df_usDollar.drop(labels=['date_time'], axis=1, inplace=True)
df_top.drop(labels=['date_time'], axis=1, inplace=True)
df_pttep.drop(labels=['date_time'], axis=1, inplace=True)

df = pd.concat(objs=[df_gold, df_ptt, df_pttep,df_top,df_usDollar], axis=1)
df.set_index('date_time', inplace=True)

In [34]:
df

,gold_price,ptt_price,pttep_price,top_price,usDollar_price
date_time,,,,,
2015-01-05,1353.5,31.5,107.00,42.75,91.38
2015-01-06,1362.2,32.3,107.00,43.00,91.50
2015-01-07,1353.6,33.4,106.50,44.25,91.89
2015-01-08,1351.6,34.0,110.00,45.75,92.37
2015-01-09,1357.4,33.8,112.50,46.00,91.94
...,...,...,...,...,...
2020-12-26,1880.7,42.0,96.50,54.00,90.32
2020-12-27,1880.7,42.0,96.50,54.00,90.32
2020-12-28,1877.9,41.0,95.00,51.25,90.34


In [36]:
df.to_csv('data.csv')